# Real-Time Face Detection with OpenCV DNN (Webcam)

This notebook demonstrates **real-time face detection** using OpenCV's Deep Neural Network (DNN) module and your system’s webcam.

---

### 🔍 Why CNN?

We use a CNN-based face detection model trained using the SSD (Single Shot Multibox Detector) framework.

- The model predicts **bounding boxes and confidence scores** in a single pass.
- It’s based on a **ResNet-10** backbone, a lightweight convolutional neural network.
- This allows fast and efficient real-time face detection, even on CPU.

---

### 🧠 Files Used
- `deploy.prototxt`: Defines CNN architecture.
- `res10_300x300_ssd_iter_140000_fp16.caffemodel`: Pre-trained CNN weights.

Make sure both files are in your working directory.


## Step 1: Import Libraries

- `cv2`: OpenCV library for video capture, DNN operations, and drawing.
- `sys`: Allows reading command-line arguments (e.g., video file paths), but optional in notebooks.


In [ ]:
import cv2
import sys

## Step 2: Set Up Video Source

- `cv2.VideoCapture(0)` opens the default webcam.
- You can also provide a path to a video file via command line (`sys.argv`).


In [18]:
# Use the default camera (0) or a video path if passed via command line
s = 0
if len(sys.argv) > 1:
    s = sys.argv[1]

source = cv2.VideoCapture(s)


## Step 3: Load CNN Model and Define Parameters

- Load the model architecture and weights.
- Set preprocessing parameters for the CNN:
  - Input size: `300x300`
  - Mean subtraction: `[104, 117, 123]` (standard for this model)
  - Confidence threshold: only show detections with score > 0.7


In [19]:
win_name = 'Camera Preview'
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

# Load the pre-trained CNN face detector
net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

# Model input size and settings
in_width = 300
in_height = 300
mean = [104, 117, 123]
conf_threshold = 0.7


## Step 4: Run Real-Time Detection Loop

- The loop runs until you press the **Esc key** (`cv2.waitKey(1) != 27`).
- Each frame is flipped horizontally (`cv2.flip(frame, 1)`) for a selfie effect.
- We convert the frame to a blob, pass it to the CNN, and extract detections.
- For detections with confidence > 0.7:
  - Bounding box is drawn.
  - Confidence label is shown above the box.


In [20]:
while cv2.waitKey(1) != 27:  # Press 'Esc' to exit
    has_frame, frame = source.read()
    if not has_frame:
        break

    # Mirror the frame (like a selfie view)
    frame = cv2.flip(frame, 1)
    frame_height, frame_width = frame.shape[:2]

    # Create blob for CNN input
    blob = cv2.dnn.blobFromImage(frame, 1.0, (in_width, in_height), mean, swapRB=False, crop=False)
    net.setInput(blob)

    # Run forward pass to detect faces
    detections = net.forward()

    # Loop through detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frame_width)
            y1 = int(detections[0, 0, i, 4] * frame_height)
            x2 = int(detections[0, 0, i, 5] * frame_width)
            y2 = int(detections[0, 0, i, 6] * frame_height)

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw confidence label
            label = f"Confidence: {confidence:.4f}"
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

            cv2.rectangle(frame, (x1, y1 - label_size[1]),
                                (x1 + label_size[0], y1 + base_line),
                                (255, 255, 255), cv2.FILLED)
            cv2.putText(frame, label, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    # Measure and display inference time
    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0))

    # Show the result in a window
    cv2.imshow(win_name, frame)

# Cleanup after loop ends
source.release()
cv2.destroyWindow(win_name)


## Step 5: Show Output and Release Resources

- Display the frame with bounding boxes and **inference time**.
- When done (after pressing Esc), release the video stream and close the window.

### 💡 Tip
In notebooks, displaying OpenCV windows can be tricky. For a better notebook experience, consider writing the frame to disk or converting it to a format viewable with `matplotlib`.
